In [32]:
from pathlib import Path
# folder with data
folder = Path('C:/Users/rigter/Desktop/Jupyter workspace/231026 SB Pietro/proper_measurements/')

Wave_Emission=700

## Define custom function

In [33]:

#%% importing packages, defining functions
import pandas as pd
import numpy as np
# from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

import matplotlib.gridspec as gridspec

cmap = plt.get_cmap('magma')

##%% definition of the big master data_import function, now for SPARS

def data_import(filename): 
    #function to open the data file, meant to work for different loops and formats
    text = open(filename, "r").readlines()
    k = 0
    i_data = 0
    while i_data == 0: #find where the different parts of the data file start
        k = k + 1
        if text[k][:6] == 'Index\t': #start of actual data
            i_data = k
            
    df = pd.read_csv(filename, delimiter = '\t', skiprows = i_data, index_col = 0)

    #remove phase data
    #changing some annoying column names
    for col in df.columns:
        if 'Aotf' in col:
            newname = 'wl'
        elif 'Piezo' in col:
            newname = col[6]
        elif 'Magnitude' in col:
            newname = col[0]
        elif 'Flip' in col:
            newname = 'LPSP'  ##value = 1 for SP, 2 for LP 
        else:
            newname = col
        df.rename(columns = {col: newname}, inplace = True)
        
        if 'Phase' in col:
            df.drop(col, axis = 1, inplace = True)
        if 'FocusC' in col:
            df.drop(col, axis = 1, inplace = True)  
      
    #taking the average of repeats and building a dataframe with the standard deviations
    if 'Repeat [x]' in df.columns:
        if 'X' and 'LPSP' and 'wl' in df.columns:
            # print('X and LPSP')
            df_sd = df.groupby(['wl', 'X', 'LPSP'], as_index = False).std()
            df = df.groupby(['wl', 'X', 'LPSP'], as_index = False).mean()
        elif 'X' and 'wl' in df.columns:
            # print('X')
            df_sd = df.groupby(['wl', 'X'], as_index = False).std()
            df = df.groupby(['wl', 'X'], as_index = False).mean()
        elif 'wl' in df.columns:
            df_sd = df.groupby(['wl'], as_index = False).std()
            df = df.groupby(['wl'], as_index = False).mean()
        else: 
            df_sd = df.groupby(['LPSP'], as_index = False).std()
            df = df.groupby(['LPSP'], as_index = False).mean()
            print('Carefull we are only selecting on the filter value')
        df.drop('Repeat [x]', axis = 1, inplace = True)
        df_sd.drop('Repeat [x]', axis = 1, inplace = True)
    else:
        df_sd = pd.DataFrame({})
        for col in df.columns:
            df_sd[col] = []
    #normalize to beam monitor
    
    if 'R' in df.columns: 
        # print('yes')
        df['RbyM'] = df.R / df.M
        df_sd['RbyM'] = abs(df.RbyM) * np.sqrt((df_sd.R/df.R)**2 \
                                                + (df_sd.M/df.M)**2)
            
    if 'I' in df.columns:
        df['IbyM'] = df.I / df.M
        df_sd['IbyM'] = abs(df.IbyM) * np.sqrt((df_sd.I/df.I)**2 \
                                                  + (df_sd.M/df.M)**2)
               
    # df = df.drop(['Index'], axis = 1)
    # df_sd = df_sd.drop(['Index', 'Repeat'], axis = 1)
    
    return df, df_sd

def abscalculator(sampleSP, mirror, miss):
    df = pd.DataFrame({'X': sampleSP.X,
                       'Z': sampleSP.Z,
                       'AR': (sampleSP.RbyM/mirror),
                       'AIS': (sampleSP.ISbyM/miss),
    })
    df['A'] = 1 - df.AR - df.AIS
    Amap = df.pivot(index = 'Z', columns = 'X', values = 'A')
    return df, Amap 



# Define responsisvity of the detector 

In [34]:
#convert responsivity data from [A/W] to [A/photon flux]

h = 6.6260e-34 #(J s) planck 
c = 2.9979e8 #(m/s) speed of light 


#ratio of generated photocurrent to incident power (A/W)
responsivity = pd.read_csv("./DataPLQY/responsivity.csv")   

#the energy of photons (J)    
responsivity['Eph'] = (h * c * 1e9)/responsivity.wavelength

responsivity['ampsbyph'] = responsivity.Eph * responsivity.responsivity

#interpolating responsivity to get entry for each Lex
interp_df = pd.DataFrame({'wavelength': np.arange(200, 1101, 2)})
resp = pd.merge(interp_df, responsivity, on = 'wavelength', how = 'left')

resp = resp.interpolate()

#selecting resp at em 

resp_em = resp[(resp.wavelength == Wave_Emission)].mean()


# Load data

In [35]:

# importing all reference files, defining responsivity at the right wavelength

PD_em, PD_em_std = data_import('./DataPLQY/20240117_165748_PDcal_em/result.txt')
PD_ex, PD_ex_std = data_import('./DataPLQY/20240117_165611_PDcal/result.txt')

#place proper ex and em wls in 770/780 and 550 spots
resp_em = responsivity[(responsivity.wavelength == 720) ^ (responsivity.wavelength == 780)].mean()
resp_ex = responsivity[responsivity.wavelength == 450].mean()

phflux_ex = PD_ex.IbyM.mean() / resp_ex.ampsbyph
phflux_em = PD_em.IbyM.mean() / resp_em.ampsbyph



mirror_em, mirror_em_std  = data_import('./DataPLQY/20240117_164734_LP_mirror/result.txt')
mirror_ex,mirror_ex_std = data_import('./DataPLQY/20240117_164456_SP_mirror/result.txt') 

miss_em,miss_em_std = data_import('./DataPLQY/20240117_163018_LP_miss/result.txt')
miss_ex,miss_em_std = data_import('./DataPLQY/20240117_160312_SP_miss/result.txt')

Reff_em = phflux_em / mirror_em.RbyM.mean()
ISeff_em = phflux_em / miss_em.IbyM.mean()

Carefull we are only selecting on the filter value
Carefull we are only selecting on the filter value
Carefull we are only selecting on the filter value
Carefull we are only selecting on the filter value
Carefull we are only selecting on the filter value
Carefull we are only selecting on the filter value


In [36]:



#%% importing unencapsulated sample map, calculating absorptance

def abs_PLQY_calc(sample_file, miss_file_em, miss_file_ex):
    
    miss_em = data_import(miss_file_em)
    miss_ex = data_import(miss_file_ex)
    ISeff_em = phflux_em / miss_em.ISbyM.mean()
    
    sample_LP, sample_SP = data_import(sample_file) 
    sample_abs = abscalculator(sample_SP, mirror_ex.RbyM.mean(), miss_ex.IbyM.mean())
        
    R_photons_em = sample_LP.RbyM * Reff_em
    IS_photons_em = sample_LP.IbyM * ISeff_em

    sample_PL = R_photons_em + IS_photons_em

    df = sample_abs[0].drop(['AIS', 'AR'], axis = 1)
    df['absflux'] = phflux_ex * df.A
    df['PLflux'] = sample_PL
    df['PLQY'] = 100 * df.PLflux/df.absflux
    return df 


#%%    

Sample = abs_PLQY_calc('./DataPLQY/20240117_145209_PLQY map/result.txt')

#%%

fig = plt.imshow(Sample.pivot(index = 'Z', columns = 'X', values = 'PLQY'))

plt.colorbar(fig, label = 'PLQY')

plt.show()

TypeError: abs_PLQY_calc() missing 2 required positional arguments: 'miss_file_em' and 'miss_file_ex'